In [1]:
import azureml.core
from azureml.core import Workspace

print(azureml.core.VERSION)
ws = Workspace.from_config()
print(ws.name,ws.location,ws.resource_group,sep='\t')

1.7.0
chatbot	westus	msinteldl


In [2]:
experiment_name = 'first-azure-ml'

from azureml.core import Experiment
exp = Experiment(workspace=ws,name=experiment_name)

In [3]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
#from azureml.train.dnn import TensorFlow

batch_size = 320
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

print(keras.__version__)

# 学習用データセット（画像（x_train）＆正解ラベル（y_train））および検証用データセット（画像（x_test）＆正解ラベル（y_test））の読み込み

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 学習用＆検証用正解ラベルをOne-Hot表現へ変換

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 学習用＆検証用画像データを標準化（0~1の値に変換）します

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# CNNモデルの構築
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# Log total number of iterations
data_augmentation = False
epochs = 5
for i in range(epochs):
    #Azure ML Serviceへのロギング開始
    run = exp.start_logging()

    # run.log("data_augmentation",data_augmentation) # Azure ML Serviceへのロギング

    # 学習処理
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=1,
                validation_data=(x_test, y_test),
                shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(x_test, y_test), workers=4)
        
    # モデルを読み込み、推論実施。モデルの精度を測る。
    #model.load_weights(model_path)
    scores = model.evaluate(x_test, y_test, verbose=1)
    run.log('Test loss', scores[0])# Azure ML Serviceへのロギング
    run.log('Test accuracy', scores[1])# Azure ML Serviceへのロギング


    # モデル（とその重み）の保存

    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)

    # run.upload_file(name = 'outputs/' + model_name, path_or_stream = model_path)# Azure ML Serviceへのロギング（ファイルのアップロード）

    # Complete tracking and get link to details
    run.complete()# Azure ML Serviceへのロギング終了
print("Run completed")

Using TensorFlow backend.


2.3.1
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 3s 335us/step
Saved trained model at /mnt/batch/tasks/shared/LS_root/mounts/clusters/basecomputing/code/Users/ego.visionary/handon/saved_models/keras_cifar10_trained_model.h5 
Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 3s 341us/step
Saved trained model at /mnt/batch/tasks/shared/LS_root/mounts/clusters/basecomputing/code/Users/ego.visionary/handon/saved_models/keras_cifar10_trained_model.h5 
Not using data augmentation.
Train on 50000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 3s 339us/step
Sav

In [6]:
experiment_name = 'resnet50'

from azureml.core import Experiment
exp = Experiment(workspace=ws,name=experiment_name)

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from tensorflow.keras import optimizers, models
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import applications
from tensorflow.keras import backend as K
import tensorflow as tf
#from azureml.train.dnn import TensorFlow

batch_size = 320
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

print(keras.__version__)

# 学習用データセット（画像（x_train）＆正解ラベル（y_train））および検証用データセット（画像（x_test）＆正解ラベル（y_test））の読み込み

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 学習用＆検証用正解ラベルをOne-Hot表現へ変換

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 学習用＆検証用画像データを標準化（0~1の値に変換）します

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


# CNNモデルの構築
# Initialize mobilenet with transfer learning
base_model = applications.ResNet50(weights='imagenet', 
                                include_top=False, 
                                input_shape=x_train.shape[1:])

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)
# and a dense layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Resnet50 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
model.summary()


# Log total number of iterations
data_augmentation = False

epochs = 10
#run.log("data_augmentation",data_augmentation) # Azure ML Serviceへのロギング
for i in range(epochs):
    #Azure ML Serviceへのロギング開始
    run = exp.start_logging()

    # 学習処理
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=1,
                validation_data=(x_test, y_test),
                shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None)

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(x_test, y_test), workers=4)
        
    # モデルを読み込み、推論実施。モデルの精度を測る。
    #model.load_weights(model_path)
    scores = model.evaluate(x_test, y_test, verbose=1)
    run.log('Test loss', scores[0])# Azure ML Serviceへのロギング
    run.log('Test accuracy', scores[1])# Azure ML Serviceへのロギング
    run.complete()# Azure ML Serviceへのロギング終了


# モデル（とその重み）の保存

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

#run.upload_file(name = 'outputs/' + model_name, path_or_stream = model_path)# Azure ML Serviceへのロギング（ファイルのアップロード）

# Complete tracking and get link to details

print("Run completed")

2.3.1
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_____________